In [1]:
### IMPLEMENTATION - NOVADESK AI CUSTOMER SUPPORT SYSTEM BLUEPRINT

# AGENTS
# Agent 1: The Orchestrator 
# Agent 2: The Triage/Intake Agent - issue category classification 
# Agent 3: The Metrics Agent - metrics of issues recorded so far (simulated)
# Agent 4: The internal knowledge agent - issues recorded (simulated)
# Agent 5: The RAG Agent - general docs about policies, products (simulated)
# Agent 6: The Response Formatter Agent - format final response 

# Response formatter gives it back to orchestrator which talks to the user (or gives it directly to the user)

# needs two secrets GOOGLE_API_KEY and GCP_SERVICE_ACCOUNT_KEY for GC Access to service account (RAG)

# WORKFLOW:
# a)The Orchestrator receives the user conversation
# b)The Orchestrator Agent calls the Triage/Intake Agent
# c)The Triage Agent classifies the message and extracts category
# d)The Triage Agent calls the combination of internal knowledge agent and business metrics agent
# if the issue is about some problem the customer has as this has information of various issues
# e)If the customer has general questions about policies and products, and if the previous step does not have
# an answer, it goes to the RAG agent
# f)The orchestrator then calls the response formatter which uses all that is available so far
# g)The Orchestrator/Response formatter passes the response back to the user as is.

# customer -> orchestrator ---------------------> triage Agent
#   |                 <------------------------------- |
#   |                                                  
#   |            |   -----> internal knowledge agent, business metrics agentRAG Agent
#   |            <---------------------------|
#   |           
#   |            |   [CONDITIONAL]-----> RAG Agent 
#   |                  <----------------|
#   |                     
#   |           | ----------------------------> Response Formatter Agent
#   |                                                 |
#   |                                                 |
#   |<------------------------------------------------|


In [2]:
#  ============ ARCHITECTURE CRITERIA MET FOR CAPSTONE ===============

# 1. Multi-agent system, including
## Agent powered by an LLM
## Parallel agents

# 2. Tools, including:
## custom tools

# 3.Sessions & Memory
## Sessions & state management (e.g. InMemorySessionService)

# 4.Context engineering (e.g. context compaction)

# 5. Observability: Logging

# 6. A2A Protocol

# 7. Evaluation - Sample Manual Tests to check agents invoked


In [3]:
# Set up environment with model key and get credentials dictionary for service account
import os
from kaggle_secrets import UserSecretsClient
import json
from google.oauth2 import service_account

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

try:
    gcp_key_json = UserSecretsClient().get_secret("GCP_SERVICE_ACCOUNT_KEY")
    
    # Parse the JSON string
    credentials_dict = json.loads(gcp_key_json)
    
    # Create credentials object
    credentials = service_account.Credentials.from_service_account_info(
        credentials_dict,
        scopes=['https://www.googleapis.com/auth/cloud-platform']
    )
    print("✅ GCP Service account key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GCP_SERVICE_ACCOUNT_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.
✅ GCP Service account key setup complete.


In [4]:
# minimal needed imports (ADK and otherwise)
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import AgentTool
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.agents import ParallelAgent

import pandas as pd
import uuid

# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

In [5]:
# Retry config for retry with exponential backoff.
from google.genai import types
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [6]:
MODEL="gemini-2.5-flash-lite"

In [7]:
# ============ AGENT 5: RAG Agent which is setup using VertexAI and A2A (just for educational purposes) ================
# ==== SIMULATED SMALL TOY DOCUMENTS OF A HEALTHCARE COMPANY STORED IN GCP BUCKET IN A CORPUS
PROJECT_ID = credentials_dict['project_id']  # Extract from the credentials
LOCATION = "us-east1"
CORPUS_NAME = 'projects/adk-rag-customer-agent/locations/us-east1/ragCorpora/6917529027641081856'

#  remote agent code
rag_agent_code = f'''
import vertexai
import json
from google.oauth2 import service_account
from google.adk.models.google_llm import Gemini
from google.adk.agents import LlmAgent
from google.genai import types
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from vertexai.preview import rag
from kaggle_secrets import UserSecretsClient

gcp_key_json = UserSecretsClient().get_secret("GCP_SERVICE_ACCOUNT_KEY")
    
# Parse the JSON string
credentials_dict = json.loads(gcp_key_json)

# Create credentials object
credentials = service_account.Credentials.from_service_account_info(
    credentials_dict,
    scopes=['https://www.googleapis.com/auth/cloud-platform']
)

# Initialize Vertex AI with passed parameters
vertexai.init(project="{PROJECT_ID}", location="{LOCATION}", credentials=credentials)

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

def query_rag_corpus(
    query_text: str,
) -> dict:
    """
    Queries a specific RAG corpus and returns relevant information.
    
    Args:
        query_text: The question or query to search for    
    Returns:
        Dictionary containing retrieved contexts and metadata
    """
    try:
        # Configure retrieval settings
        rag_retrieval_config = rag.RagRetrievalConfig(
            top_k=5,
            filter=rag.Filter(vector_distance_threshold=0.5),
        )
        
        # Perform retrieval query
        response = rag.retrieval_query(
            rag_resources=[
                rag.RagResource(
                    rag_corpus="{CORPUS_NAME}",
                )
            ],
            text=query_text,
            rag_retrieval_config=rag_retrieval_config,
        )
        
        # Extract relevant contexts
        contexts = []
        for context in response.contexts.contexts:
            source_display_name = getattr(context, 'source_display_name', None)
            score = getattr(context, 'score', None)

            contexts.append({{
                "text": context.text,
                # Use 'score' for the similarity/distance metric
                "score": score, 
                # Human-readable filename
                "source_display_name": source_display_name,
            }})
        
        return {{
            "status": "success",
            "query": query_text,
            "contexts": contexts,
            "num_results": len(contexts)
        }}
        
    except Exception as e:
        return {{
            "status": "error",
            "error_message": str(e),
            "query": query_text
        }}

# Create the ADK Agent
rag_agent = LlmAgent(
    model=Gemini(
        model="{MODEL}",
        retry_options=retry_config
    ),
    name='rag_agent',
    instruction="""
    You are a helpful AI assistant with access to a knowledge corpus.
    
    When a user asks a question:
    1. Use the query_rag_corpus tool to retrieve relevant information from the corpus
    2. Analyze the retrieved contexts carefully
    3. Generate a comprehensive answer based on the retrieved information
    4. ALWAYS cite your sources at the end of the answer using the format: [Source Document Name].
    5. If the retrieved information doesn't fully answer the question, say so
    
    Format your responses clearly and include citations.
    """,
    tools=[query_rag_corpus],
    # stored in this key for access in the agent pipeline
    output_key="rag_result"
)

app = to_a2a(
    rag_agent, port=8001  # Port where this agent will be served
)
'''

In [8]:
## server creation
import subprocess, requests, time
import threading

# Write the rag agent to a temporary file
with open("/tmp/rag_server.py", "w") as f:
    f.write(rag_agent_code)

print("📝 RAG agent code saved to /tmp/rag_server.py")

# Start uvicorn server in background
# Note: We redirect output to avoid cluttering the notebook
server_process = subprocess.Popen(
    [
        "uvicorn",
        "rag_server:app",  # Module:app format
        "--host",
        "localhost",
        "--port",
        "8001",
    ],
    cwd="/tmp",  # Run from /tmp where the file is
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},  # Pass environment variables (including GOOGLE_API_KEY)
)

print("🚀 Starting rag server...")
print("   Waiting for server to be ready...")

# Wait for server to start (poll until it responds)
max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8001/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print(f"\n✅ RAG Agent server is running!")
            print(f"   Server URL: http://localhost:8001")
            print(f"   Agent card: http://localhost:8001/.well-known/agent-card.json")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️  Server may not be ready yet. Check manually if needed.")

# Store the process so we can stop it later
globals()["rag_server_process"] = server_process

📝 RAG agent code saved to /tmp/rag_server.py
🚀 Starting rag server...
   Waiting for server to be ready...
....
✅ RAG Agent server is running!
   Server URL: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json


In [9]:
## Fetch the agent card from the running server
try:
    response = requests.get(
        "http://localhost:8001/.well-known/agent-card.json", timeout=5
    )

    if response.status_code == 200:
        agent_card = response.json()
        print("📋 RAG Agent Card:")
        print(json.dumps(agent_card, indent=2))

        print("\n✨ Key Information:")
        print(f"   Name: {agent_card.get('name')}")
        print(f"   Description: {agent_card.get('description')}")
        print(f"   URL: {agent_card.get('url')}")
        print(f"   Skills: {len(agent_card.get('skills', []))} capabilities exposed")
    else:
        print(f"❌ Failed to fetch agent card: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching agent card: {e}")
    print("   Make sure the RAG server is running (previous cell)")

📋 RAG Agent Card:
{
  "capabilities": {},
  "defaultInputModes": [
    "text/plain"
  ],
  "defaultOutputModes": [
    "text/plain"
  ],
  "description": "An ADK Agent",
  "name": "rag_agent",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "\n    I am a helpful AI assistant with access to a knowledge corpus.\n    \n    When a user asks a question:\n    1. Use the query_rag_corpus tool to retrieve relevant information from the corpus\n    2. Analyze the retrieved contexts carefully\n    3. Generate a comprehensive answer based on the retrieved information\n    4. ALWAYS cite my sources at the end of the answer using the format: [Source Document Name].\n    5. If the retrieved information doesn't fully answer the question, say so\n    \n    Format my responses clearly and include citations.\n    ",
      "id": "rag_agent",
      "name": "model",
      "tags": [
        "llm"
      ]
    },
    {
      "description": "Queries a sp

In [10]:
## Create a RemoteA2Agent that connects to the RAG Agent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)
remote_rag_agent = RemoteA2aAgent(
    name="rag_agent",
    description="Remote rag agent that provides information.",
    # Point to the agent card URL - this is where the A2A protocol metadata lives
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote RAG Agent proxy created!")
print(f"   Connected to: http://localhost:8001")
print(f"   Agent card: http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}")
print("   Other agents can now use this like a local agent")

✅ Remote RAG Agent proxy created!
   Connected to: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json
   Other agents can now use this like a local agent


In [11]:
# ============= AGENT 2: Triage Agent =========================
triage_agent = LlmAgent(
    model=Gemini(model=f"{MODEL}", retry_options=retry_config),
    name="triage_agent",
    description="Analyzes customer query to extract category of the query.",
    instruction="""
    You are a query analysis specialist. Your ONLY job is to analyze the customer's message - DO NOT try to answer their question.
    
    For the given customer query, perform the following analysis:
    
    1. Category classification:
       - "login" - Login related query
       - "performance" - Performance related query
       - "billing" - Billing related query
       - "feedback" - any general statements from the customer
       - "other" - Anything else that is not in the above categories
    
    IMPORTANT: Return ONLY a JSON object with your analysis. Do not answer the customer's question.
    
    Format:
    {
        "query_category": "<category>"
    }
    """,
    output_key="query_category"
)
print("✅ Triage Agent created!")

✅ Triage Agent created!


In [12]:
# Knowledge base of simulated issues
faqs_df = pd.DataFrame(
    [
        {
            "doc_id": 1,
            "category": "billing",
            "question": "Why did I get charged twice for my subscription?",
            "answer": (
                "Customers sometimes see two similar charges when one is an "
                "authorization hold. The duplicate pending charge usually "
                "drops within 3–5 business days. If both charges settle, the "
                "customer is eligible for a refund."
            ),
        },
        {
            "doc_id": 2,
            "category": "billing",
            "question": "How do customers request a refund for incorrect charges?",
            "answer": (
                "Customers should contact support within 30 days of the "
                "transaction and provide the last 4 digits of the card, date, "
                "and amount. Support submits a refund request which is "
                "reviewed within 2–3 business days."
            ),
        },
        {
            "doc_id": 3,
            "category": "performance",
            "question": "What to do when dashboards are slow?",
            "answer": (
                "Ask the customer for the affected workspace, time window, "
                "and approximate user count. Check current incident status, "
                "then share any known outage or suggest limiting filters and "
                "refreshing the page."
            ),
        },
        {
            "doc_id": 4,
            "category": "login",
            "question": "How to troubleshoot login issues?",
            "answer": (
                "Confirm the email address, verify that the account is active, "
                "and ask the user to reset the password. If the issue persists, "
                "escalate to engineering with recent login timestamps."
            ),
        },
    ]
)

In [13]:
# Tool to search internal FAQ for issues
def search_faq_tool(query_category: str) -> dict:
    """
    Search the internal FAQ / policy knowledge base for relevant entries.

    Use this when you need official guidance, policies, or FAQ-style answers
    to support a customer ticket.

    Args:
        query_category: High level category such as "billing", "performance", "login".

    Returns:
        A dictionary with:
            status: <status of the lookup>
            category: <the category used>
            matches: <matched entries>
    """
    # scores: List[Dict[str, Any]] = []
    cat = query_category.lower().strip()
    subset = faqs_df[faqs_df["category"] == cat]

    if not subset.empty: 
        return {
        "status": "success",
        "category": cat,
        "data": subset.to_dict(orient="records"),
    }
    else:
        return {
        "status": "failed",
        "category": cat,
        "data": None
    }
        

In [14]:
# ========= AGENT 3: Knowledge Agent  ========= 
knowledge_agent = LlmAgent(
    name="KnowledgeAgent",
    model=Gemini(model=f"{MODEL}", retry_options=retry_config),
    description=(
        "Looks up internal FAQ and policy text to help answer the support ticket."
    ),
    instruction=(
        "You are the knowledge agent in an enterprise support copilot.\n"
        "Your job is to lookup in the internal knowledge base using the search_faq_tool to retrieve the most relevant entries specific to a category.\n"
        "In your final answer, briefly summarize the key guidance and policies that are relevant.\n"
        "Keep the answer short."
    ),
    tools=[search_faq_tool],
    output_key="faq_context",  # stored in session.state["faq_context"]
)

In [15]:
# Business metrics for simulated issues
metrics_df = pd.DataFrame(
    [
        {
            "date": "2025-10-29",
            "category": "billing",
            "metric_name": "duplicate_charge_tickets_last_7d",
            "value": 4,
        },
        {
            "date": "2025-11-01",
            "category": "billing",
            "metric_name": "duplicate_charge_tickets_last_7d",
            "value": 10,
        },
        {
            "date": "2025-11-01",
            "category": "billing",
            "metric_name": "refund_requests_last_7d",
            "value": 7,
        },
        {
            "date": "2025-11-01",
            "category": "performance",
            "metric_name": "slow_dashboard_tickets_last_7d",
            "value": 15,
        },
        {
            "date": "2025-11-01",
            "category": "login",
            "metric_name": "login_issue_tickets_last_7d",
            "value": 5,
        },
    ]
)

In [16]:
# Tool to search business metrics for issues
def business_metrics_tool(query_category: str) -> dict:
    """
    Return recent business metrics relevant to a ticket category.

    Use this when you need to mention trends, spikes, or volume for billing,
    login, or performance issues.

    Args:
        query_category: High level category such as "billing", "performance", "login".

    Returns:
        A dictionary with:
            status: <status of the lookup>
            category: <the category used>
            matches: <matched entries>
    """
    cat = query_category.lower().strip()
    subset = metrics_df[metrics_df["category"] == cat]

    if not subset.empty: 
        return {
        "status": "success",
        "category": cat,
        "matches": subset.to_dict(orient="records"),
    }
    else:
        return {
        "status": "failed",
        "category": cat,
        "matches": None
    }

In [17]:
#  ========= AGENT 4: Metrics Agent  ========= 
metrics_agent = LlmAgent(
    name="MetricsAgent",
    model=Gemini(model=f"{MODEL}", retry_options=retry_config),
    description="Looks up recent business metrics for this type of ticket.",
    instruction=(
        "You are the metrics analysis agent for the support team.\n"
        "Your job is to lookup in the using the business_metrics_tool to retrieve the most relevant entries specific to a category.\n"
        "Keep the answer short.""In your final answer, summarize any relevant volumes or trends that might be useful for an internal note.\n"
        "For example, note if there is a spike in similar tickets over the last week.\n"
        "Keep the answer short, one or two sentences."
    ),
    tools=[business_metrics_tool],
    output_key="metrics_context",  # stored in session.state["metrics_context"]
)

In [18]:
# ========== AGENT 3/4 Combined - define the parallel agent ==========
faq_metrics_parallel_agent = ParallelAgent(
    name="faq_metrics_parallel_agent",
    sub_agents=[knowledge_agent, metrics_agent],
)

In [46]:
# =========== AGENT 6: Response Formatter Agent ==================
customer_response_agent = LlmAgent(
    name="customer_response_agent",
    model=Gemini(model=f"{MODEL}", retry_options=retry_config),
    description=(
        "Provide the final reply to the customer and create internal notes."
    ),
    instruction=(
        "1. You will provide the final response to the user for a support query in a specific layout.\n"
        
        "2. You must draft both (a) a customer-facing reply and (b) internal notes for the support team.\n"
        
        "3. You have three pieces of context stored in session state:\n"
        "- faq_context: {faq_context?}\n"
        "- metrics_context: {metrics_context?}\n"
        "- rag_context: {rag_context?}\n"
        "Use these to produce your answer in Markdown with two sections:\n"
        "### Reply\n"
        "<ResponseGuidelines>"
        "- Always be warm and professional"
        "- Match the customer's tone (empathetic if upset, friendly if casual)"
        "- Keep introductions and greetings natural and conversational"
        "</ResponseGuidelines>"
        "Write a short reply in plain language that addresses the issue, next steps, "
        "and any relevant expectations (like timelines).\n"
        
        "### Internal Notes\n"
        "<internalNotes>"
        "Summarize the ticket in a few bullet points, including:\n"
        "- Suggested tags (for example: billing, duplicate_charge).\n"
        "- Priority.\n"
        "- Any patterns or metrics worth noting.\n"
        "- Recommended follow up action or reminder.\n"
        "</internalNotes>"
        "Do not expose internal policies or metrics directly to the customer.\n"
    ),
    # We could also set output_key if we want to store the final answer in state.
    output_key="final_detailed_response"
)

In [47]:
# =========== AGENT 1: Orchestrator Agent ==================
orchestrator_agent = LlmAgent(
    model=Gemini(model=f"{MODEL}", retry_options=retry_config),
    name="unified_support_agent",
    description="Unified customer support agent that handles all queries",
    instruction="""
    You are a customer support agent with access to conversation history and tools.
    
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    🧠 CRITICAL: You have FULL conversation history. Use it!
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    DECISION TREE:
    
    1️. Can you answer from conversation history?
       Examples:
       - User introduced themselves → Remember their name
       - User mentioned order number → Remember it
       - User described their issue → Remember the context
       
       → If YES: Answer directly. DO NOT use any tools.
        
    2. Is this a NEW support query needing documentation/data?
       - Product questions not answered yet
       - Technical issues requiring troubleshooting steps
       - Policy/pricing information
       - Account lookups
       
       → If YES: Execute support workflow with tools.
    
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    SUPPORT WORKFLOW (Only for #2):
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    Step 1: Call triage_agent (categorize the issue)
    Step 2: Call faq_metrics_parallel_agent (search FAQ & metrics)
    Step 3: Evaluate the information obtained:
            - If sufficient → Jump to Step 5
            - If insufficient → Call rag_agent (deep documentation search)
    Step 4: [Optional] Call rag_agent if needed
    Step 5: [MANDATORY] Call final_response_agent to provide a user reply.

    🛑 CRITICAL INSTRUCTION:
    Once you gather information from the tools (FAQ or RAG), you MUST NOT output the answer directly as text.
    
    You MUST call the 'final_response_agent' tool and pass the information to it.
    
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    EXAMPLES:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    ✅ CORRECT:
    User: "I'm Sarah"
    You: "Hello Sarah! How can I help you?" [NO TOOLS]
    
    User: "What's my name?"
    You: "Your name is Sarah." [NO TOOLS - from history]
    
    User: "How do I reset my password?"
    You: [Execute full workflow with tools]
    
    ❌ WRONG:
    User: "I'm Sarah"
    You: "Hello Sarah!"
    
    User: "What's my name?"  
    You: [Calls triage_agent] ← WRONG! You already know!
    
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    Remember: Tools are for getting NEW information, not information 
    the user already gave you in this conversation!
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    """,
    tools=[
        AgentTool(agent=triage_agent),
        AgentTool(agent=faq_metrics_parallel_agent),
        AgentTool(agent=remote_rag_agent),
        AgentTool(agent=customer_response_agent)
    ]
)

In [48]:
# Define the customer support system invocation
async def customer_support_system(
    runner_instance: Runner,
    user_query: str,
    session_name: str,
    user_id: str = "default"):
    """Enhanced version with comprehensive event logging for debugging"""

    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name
    
    # Create session
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=user_id, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=user_id, session_id=session_name
        )    
    
    # Create the user message
    test_content = types.Content(parts=[types.Part(text=user_query)])
    
    # Display query
    print(f"\n{'='*70}")
    print(f"👤 CUSTOMER QUERY: {user_query}")
    print(f"{'='*70}\n")
    
    step_count = 0
    # Run the agent with detailed event logging
    async for event in runner.run_async(
        user_id=user_id, session_id=session_name, new_message=test_content
    ):
        step_count += 1

            # # # ALTERNATIVE DEBUGGING LOGGING to show the sequence
            # for i, part in enumerate(event.content.parts):
                # # Text content
                # if hasattr(part, 'text') and part.text:
                #     text_preview = part.text[:100] + "..." if len(part.text) > 100 else part.text
                #     print(f"   📝 Text [{i}]: {text_preview}")
                
                # # Function calls
                # if hasattr(part, 'function_call') and part.function_call:
                #     fc = part.function_call
                #     print(f"   🔧 Function Call [{i}]:")
                #     print(f"      Name: {fc.name}")
                #     print(f"      Args: {fc.args}")
                
                # # Function responses
                # if hasattr(part, 'function_response') and part.function_response:
                #     fr = part.function_response
                #     print(f"   ✅ Function Response [{i}]:")
                #     print(f"      Name: {fr.name}")
                #     print(f"      Response: {fr.response}")
        # The final response from the agent contains the final user-facing text.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Assuming the final response text is in the first part
                final_text = event.content.parts[0].text
                if final_text:
                    print(f"\n🤖 FINAL MODEL RESPONSE: {final_text}")
    # Summary
    print(f"\n📊 EXECUTION SUMMARY:")
    print(f"   Total steps: {step_count}")

In [49]:
# Define the Runner with Compaction Option
# Setup session management - USES INMEMORYSESSION ONLY
from google.adk.apps.app import App, EventsCompactionConfig

session_service = InMemorySessionService()

customer_app = App(
    name="customer_app",
    root_agent=orchestrator_agent,
    # This is the new part!
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1,  # Keep 1 previous turn for context
    ),
)

# Step 3: Create the Runner
runner = Runner(app=customer_app, session_service=session_service)

In [50]:
# ********************** Multi-turn class definition of the NovaDesk AI customer support system ************************
class CustomerSupportSession:
    def __init__(self, runner_instance, session_id="customer_demo"):
        self.runner = runner_instance
        self.session_id = session_id
        self.turn_count = 0
        self.is_active = False
    
    async def start(self):
        """Start a new customer support session"""
        self.turn_count = 0
        self.is_active = True
        
        print("╔" + "═" * 58 + "╗")
        print("║  Customer Support Agent - Interactive Session          ║")
        print("║  Commands: 'quit'/'exit'/'bye' to end                  ║")
        print("╚" + "═" * 58 + "╝")
        print(f"Session ID: {self.session_id}\n")
        
        await self._conversation_loop()
    
    async def _conversation_loop(self):
        """Main conversation loop"""
        while self.is_active:
            try:
                user_input = input("You: ").strip()
                
                if user_input.lower() in ['quit', 'exit', 'bye', 'end']:
                    await self.end_session()
                    break
                
                if not user_input:
                    continue
                
                await self._process_message(user_input)
                
            except KeyboardInterrupt:
                print("\n")
                await self.end_session()
                break
            except Exception as e:
                print(f"✗ Error: {e}\n")
    
    async def _process_message(self, message: str):
        """Process a single message"""
        await customer_support_system(
            runner_instance=self.runner,
            user_query=message,
            session_name=self.session_id)
        self.turn_count += 1
        print()
    
    async def end_session(self):
        """End the current session"""
        self.is_active = False
        print(f"\n✓ Session ended")
        print(f"Session ID: {self.session_id}")
        print(f"Total interactions: {self.turn_count}")

In [51]:
# ********************** INVOCATION OF NOVADESK AI CUSTOMER SUPPORT SYSTEM *****************
session_id=f"order_{uuid.uuid4().hex[:8]}"
support_session = CustomerSupportSession(runner_instance=runner, session_id=session_id)
await support_session.start()

╔══════════════════════════════════════════════════════════╗
║  Customer Support Agent - Interactive Session          ║
║  Commands: 'quit'/'exit'/'bye' to end                  ║
╚══════════════════════════════════════════════════════════╝
Session ID: order_437e7b57



You:  I am curious about returns



 ### Session: order_437e7b57

👤 CUSTOMER QUERY: I am curious about returns




🤖 FINAL MODEL RESPONSE: Hello there,

I understand you're looking for information about our returns policy at Vitality Health & Fitness. I'm happy to clarify that for you!

You can return items within 30 days of your purchase date, as long as they are unopened, unused, and still in their original packaging. Please note that some items, like perishable goods, digital products, and opened nutrition supplements, cannot be returned.

To start a return, simply reach out to us at support@vitalityhealth.com with your order number. Once we receive and inspect your returned item, your refund should be processed within 5-7 business days.

If you happen to return an item between 30 and 60 days after purchase, we'll be able to issue a store credit for you.

Please let me know if you have any other questions!

📊 EXECUTION SUMMARY:
   Total steps: 9



You:  i am arun



 ### Session: order_437e7b57

👤 CUSTOMER QUERY: i am arun


🤖 FINAL MODEL RESPONSE: Hello Arun! How can I help you today?

📊 EXECUTION SUMMARY:
   Total steps: 1



You:  quit



✓ Session ended
Session ID: order_437e7b57
Total interactions: 2


In [26]:
# Illustrate that there was compaction involved if there were more than 3 conversations in the session
final_session = await session_service.get_session(
    app_name=runner.app_name,
    user_id="default",
    session_id=support_session.session_id,
)

print("--- Searching for Compaction Summary Event ---")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print("\n✅ SUCCESS! Found the Compaction Event:")
        print(f"  Author: {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print(
        "\n❌ No compaction event found. Try increasing the number of turns in the demo."
    )

--- Searching for Compaction Summary Event ---

✅ SUCCESS! Found the Compaction Event:
  Author: user

 Compacted information: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='f531f7d1-fbf9-407b-b876-bf439e9199da' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction=EventCompaction(start_timestamp=1764456665.31296, end_timestamp=1764456710.566758, compacted_content=Content(
  parts=[
    Part(
      text="""**Summary of Conversation:**

Arun has initiated a conversation with the AI agent, stating his name an

In [ ]:
# DEBUGGING VIEW OF THE SESSION - SEE THE SESSION FLOW
session = await session_service.get_session(
            app_name=runner.app_name,
            user_id="default",
            session_id=session_id
        )
print(session)

In [52]:
# LOGGINGPLUGIN BASED DEBUGGING INSPECTION: Set up debugging runner
# Adding LoggingPlugin for debugging the orchestrator agent
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)  # <---- 1. Import the Plugin

debug_runner = InMemoryRunner(
    agent=orchestrator_agent,
    plugins=[
        LoggingPlugin()
    ],  # <---- 2. Add the plugin. Handles standard Observability logging across ALL agents. 
    # See https://github.com/google/adk-python/blob/main/src/google/adk/plugins/logging_plugin.py
)

print("✅ Runner configured")

✅ Runner configured


In [53]:
# LOGGINGPLUGIN INSPECTION: Debug run the support system
print("🚀 Running orchestrator agent with LoggingPlugin...")
print("📊 Watch the comprehensive logging output below:\n")

response = await debug_runner.run_debug("I am curious about returns")

🚀 Running orchestrator agent with LoggingPlugin...
📊 Watch the comprehensive logging output below:


 ### Created new session: debug_session_id

User > I am curious about returns
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-4795fb3c-da75-4c5c-b5a4-1bd941707097
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: unified_support_agent
[logging_plugin]    User Content: text: 'I am curious about returns'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-4795fb3c-da75-4c5c-b5a4-1bd941707097
[logging_plugin]    Starting Agent: unified_support_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: unified_support_agent
[logging_plugin]    Invocation ID: e-4795fb3c-da75-4c5c-b5a4-1bd941707097
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: unifie

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: unified_support_agent
[logging_plugin]    Content: function_call: triage_agent
[logging_plugin]    Token Usage - Input: 815, Output: 20
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: dfc7e8b8-661c-443d-9a3e-08d46ab729ec
[logging_plugin]    Author: unified_support_agent
[logging_plugin]    Content: function_call: triage_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['triage_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: triage_agent
[logging_plugin]    Agent: unified_support_agent
[logging_plugin]    Function Call ID: adk-44087b45-2645-4972-8e21-a5426ce48bf7
[logging_plugin]    Arguments: {'request': 'I am curious about returns'}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-4988661c-0820-4cda-9b98-69961c6e4600
[logging_plugin]    Session ID: 0b743c94-e94f-4ccf-9041-db8270c22940
[logging_plugin]    User ID: debug_user_i

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: unified_support_agent
[logging_plugin]    Content: function_call: faq_metrics_parallel_agent
[logging_plugin]    Token Usage - Input: 869, Output: 20
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 52f00807-d0a8-44ed-89bd-f85035b38373
[logging_plugin]    Author: unified_support_agent
[logging_plugin]    Content: function_call: faq_metrics_parallel_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['faq_metrics_parallel_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: faq_metrics_parallel_agent
[logging_plugin]    Agent: unified_support_agent
[logging_plugin]    Function Call ID: adk-2ea3f0db-2086-4447-8144-1c6d6d9e6391
[logging_plugin]    Arguments: {'request': 'Returns policy'}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-29455bec-2783-4621-b7c2-0964fdaf1053
[logging_plugin]    Session ID: 66f50f14-d182-49dd-807a-c9b2b4167e

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: KnowledgeAgent
[logging_plugin]    Content: function_call: search_faq_tool
[logging_plugin]    Token Usage - Input: 242, Output: 19
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 573ff3d6-52f3-43f0-ad48-24cc4f00ca84
[logging_plugin]    Author: KnowledgeAgent
[logging_plugin]    Content: function_call: search_faq_tool
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['search_faq_tool']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: search_faq_tool
[logging_plugin]    Agent: KnowledgeAgent
[logging_plugin]    Function Call ID: adk-bcac3b1a-e3e3-4a50-ae38-7c2459d74358
[logging_plugin]    Arguments: {'query_category': 'returns'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: search_faq_tool
[logging_plugin]    Agent: KnowledgeAgent
[logging_plugin]    Function Call ID: adk-bcac3b1a-e3e3-4a50-ae38-7c2459d74358
[logging_plugin]    Result: {'status': 'fail

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: unified_support_agent
[logging_plugin]    Content: function_call: rag_agent
[logging_plugin]    Token Usage - Input: 931, Output: 16
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: ddde3585-7954-4e8e-9dfd-771d1b8b610b
[logging_plugin]    Author: unified_support_agent
[logging_plugin]    Content: function_call: rag_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['rag_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: rag_agent
[logging_plugin]    Agent: unified_support_agent
[logging_plugin]    Function Call ID: adk-b5cd07f4-4726-478d-8b64-b944d1fe0da7
[logging_plugin]    Arguments: {'request': 'Returns policy'}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-a7400c48-c018-4a48-85ac-d9ca73303816
[logging_plugin]    Session ID: 200a563a-001a-49f3-94c9-99837f137388
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    Ap

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: unified_support_agent
[logging_plugin]    Content: function_call: customer_response_agent
[logging_plugin]    Token Usage - Input: 1125, Output: 168
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 45945a2b-f6d2-4030-9c48-1950050ae2ca
[logging_plugin]    Author: unified_support_agent
[logging_plugin]    Content: function_call: customer_response_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['customer_response_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: customer_response_agent
[logging_plugin]    Agent: unified_support_agent
[logging_plugin]    Function Call ID: adk-be134477-7514-495b-805a-7f7c356e71d3
[logging_plugin]    Arguments: {'request': 'The Vitality Return & Refund Policy allows for product returns within 30 days of the original purchase date. To be eligible for a full refund, items must be in their original packaging, unopened, and unused.\